<a href="https://colab.research.google.com/github/Aztec95pa/LLM-exercise-from-google-colab/blob/main/LLM%E8%AC%9B%E5%BA%A72024_%E7%AC%AC2%E5%9B%9E%E6%BC%94%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2024年 大規模言語モデル サマースクール 第2回演習


## 演習の目的
「学習済みのLLM を（追加学習せずに）活用する技術について学ぶ」という目的のもと以下の目標を設定して講義を行いました。
- Prompting やIn-Context Learning とはなにか説明できるようになる。
- Augmented Language Model とはなにか説明できるようになる。

本演習では、Hugging Face を使った公開モデルを使った基本的なPromptingの実装、Augmented Language Models の1 種であるRetrieval Augmented Generation の実装を行います。

## 目次
- Prompting
    - 公開モデルをHuggingFace経由で使用
    - Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
    - Prompt engineering guide
- Retrieval Augmented Generation
    - Generation without Retrieval
    - Generation with gold passage
    - Retrieval の実装
    - Chunkingの実装 (補足)
    - Retrieval Augmented Generation
    - Rerankingの実装 (補足)


## Prompting
タスクに合わせてpromptを設計することで、追加の学習コストなしで大規模モデルを活用してタスクを解かせることができます。
重みにアクセスできないがAPI経由で使用できるClosed Model、重みにアクセスできるOpen Modelを用いてzero-shot / few-shot / CoT Promptingを試してみましょう。

### 公開モデルをHuggingFace経由で使用
API経由で使用できる大規模言語モデルは通常モデルの重みにはアクセスできず、自前データでの学習やモデルの分析などには限界があります。
研究機関や企業がモデルの重みを公開することもあり、1個人であっても大規模に訓練されたモデルの重みを活用して自由に使用することができます。(商用利用など特定の用途はライセンスによって制限される場合があります。)

Meta社から公開されたLlama3モデルは以下のリンクからリクエストを送信することで重みのダウンロードリンクを得ることができ、指定のライセンスのもと使用することができます。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
https://github.com/meta-llama/llama3


HuggingFaceというプラットフォーム上で公開されることも多くあり、演習ではtransformersというライブラリを使用してHuggingFace上に公開されているモデルを使用してみます。  
https://huggingface.co/  
https://huggingface.co/docs/transformers/index  
HuggingFace上に公開されているモデルをtransformersライブラリを使用しロードし、ロードしたモデルにpromptを入れて出力を確認しましょう。

Llama3モデルの使用にはライセンスの同意の上、申請が必要なので各自申請をお願いします。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
上記の申請後以下からHuggingFaceでも申請を行う(Meta社への申請のメールアドレスとHuggingFaceのアカウントで登録しているメールアドレスが一致する必要あり)  
https://huggingface.co/meta-llama/Meta-Llama-3-8B

In [7]:
# 自分のHuggingFaceアカウントと紐付ける(申請済みのアカウントでないとモデルをダウンロードできないため)、terminalの場合はhuggingface-cli loginを実行してください。
from huggingface_hub import notebook_login

notebook_login()

In [8]:
# 量子化に必要なパッケージです
!pip install bitsandbytes

In [9]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# https://huggingface.co/meta-llama/Meta-Llama-3-8B
# https://huggingface.co/meta-llama/Meta-Llama-3-8B/tree/main
# from_pretrainedの引数にモデル名を指定すると、モデルをダウンロードしてきてくれます。ダウンロードには3分ほどかかります。

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

#　メモリが足りない場合は, 量子化のために以下を有効にしてください
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B",
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [6]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# https://huggingface.co/meta-llama/Meta-Llama-3-8B
# https://huggingface.co/meta-llama/Meta-Llama-3-8B/tree/main
# from_pretrainedの引数にモデル名を指定すると、モデルをダウンロードしてきてくれます。ダウンロードには3分ほどかかります。

# Get your Hugging Face token from https://huggingface.co/settings/tokens
# and paste it below.
token = "%-rCY@MMf89@Rna"

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_auth_token=token)

#　メモリが足りない場合は, 量子化のために以下を有効にしてください
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B",
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
            use_auth_token=token  # Pass the token here as well
        )

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: meta-llama/Meta-Llama-3-8B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [10]:
prompt = "Explain about Large Language Model briefly."

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              do_sample=False, #Temperatureを有効にする場合はTrueにしてください。
              #temperature=0.6,
              #top_p=0.9
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print("\n-----生成結果-----\n", output)

  del input_ids
  del model_input
  torch.cuda.empty_cache()

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

### Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
promptingによって大規模言語モデルの出力が変化することを実感してみましょう。

JGLUEのタスクの一つであるJCommonSenseQAを例に、zero-shot / few-shot promptingの効果を確認してみます。

In [17]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ... (other code remains the same) ...

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_auth_token=token)

# Manually set the padding token
tokenizer.pad_token = tokenizer.eos_token # or '[PAD]' if you prefer
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

for i in range(3):
  model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
  input_ids = model_input["input_ids"]

  with torch.no_grad():
    result = model.generate(
                input_ids,
                max_new_tokens=300,
                # eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output.split("[問題]")[0])

    del input_ids
    del model_input
    torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""
import torch
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

for i in range(3):
  model_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
  input_ids = model_input["input_ids"]

  with torch.no_grad():
    result = model.generate(
                input_ids,
                max_new_tokens=300,
                # eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output.split("[問題]")[0])

    del input_ids
    del model_input
    torch.cuda.empty_cache()
    result = model.generate(
                input_ids,
                max_new_tokens=300,
                # eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output.split("[問題]")[0])

    del input_ids
    del model_input
    torch.cuda.empty_cache()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [15]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ... (other code remains the same) ...

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_auth_token=token)

# Manually set the padding token
tokenizer.pad_token = tokenizer.eos_token # or '[PAD]' if you prefer

# ... (rest of your code remains the same) ...

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [18]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# モデルID
model_id = "meta-llama/Meta-Llama-3-8B"

# トークナイザーのロード（必要であれば use_auth_token=token を設定）
tokenizer = AutoTokenizer.from_pretrained(model_id)  # 既に認証が済んでいる場合

# pad_token の設定（任意。多くの場合、padding=True とすれば attention_mask も生成される）
tokenizer.pad_token = tokenizer.eos_token

# 量子化の設定（メモリ節約用）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

# モデルのロード
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

# プロンプトの定義
prompt = "Explain about Large Language Model briefly."

# 注意：padding=True, truncation=True を追加
model_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
    result = model.generate(
        input_ids,
        max_new_tokens=300,
        do_sample=False,  # 決定論的生成の場合
    )
    # 入力文分をスライスして、生成された新規トークン部分のみ取得
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output)

    del input_ids
    del model_input
    torch.cuda.empty_cache()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


zero-shotでは答えが不安定で, 望ましい形式で出力されていないのに対し、few-shot promptingでは答えの単語のみ選ばれていることが確認でき、formattingの効果があることがわかります。  
次にCoT Promptingを試してみましょう。  
今回は、CoT promptingが提唱された論文中の例を試してみます。  

In [ ]:
import torch
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              # eos_token_id=terminators,
              do_sample=True,
              temperature=0.6,
              top_p=0.9,
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print("\n-----生成結果-----\n", output.split("Q:")[0])

  del input_ids
  del model_input
  torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 8




In [ ]:
prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              # eos_token_id=terminators,
              do_sample=True,
              temperature=0.6,
              top_p=0.9,
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print("\n-----生成結果-----\n", output.split("Q:")[0])

  del input_ids
  del model_input
  torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 23 dollars. 5 bagels. Each bagel costs $3. So she paid $3 * 5 = 15 dollars for the bagels. She has 23 - 15 = 8 dollars left. The answer is 8.



In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              do_sample=True,
              temperature=0.6,
              top_p=0.9,
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print("\n-----生成結果-----\n", output.split("Q:")[0])

  del input_ids
  del model_input
  torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
  Olivia has $23.  She bought five bagels for $3 each.  How much money does she have left?
$23 - 5 * $3 = $23 - $15 = $8.  She has $8 left.





「Let's think step by step」という文言によって思考の流れが出力されるようになりました。  
次にprompt中の些細な違いが出力にどのような影響を与えるかを確認してみましょう。

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              do_sample=False,
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print('\n\n############# with space in the end #############')
  print(output.split("Q:")[0], "\n\n")

  del input_ids
  del model_input
  torch.cuda.empty_cache()

prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""

model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = model_input["input_ids"]

with torch.no_grad():
  result = model.generate(
              input_ids,
              max_new_tokens=300,
              do_sample=False,
          )
  result = result[0][input_ids.shape[-1]:]
  output = tokenizer.decode(result, skip_special_tokens=True)
  print('\n\n############# with 全角space in the end #############')
  print(output.split("Q:")[0], "\n\n")

  del input_ids
  del model_input
  torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




############# with space in the end #############
 First, Olivia bought five bagels. She paid $3 for each bagel. So, she paid $3 times 5, which is $15. She has $23. She paid $15. So, she has $23 minus $15, which is $8. Olivia has $8 left.

 




############# with 全角space in the end #############
First, Olivia bought five bagels for $3 each. So, she spent $15. Then, she has $23-$15=$8 left. So, Olivia has $8 left.

 




### Prompt engineering guide
tokenizerや学習データの違いにより、使用するモデルによって、promptingの影響は異なります。  
選定したモデルによってどのような特徴があるのか、そのモデルが提唱された論文や公式ドキュメントを確認することで推し量ることができます。  
例えばOpenAI社のGPTシリーズでは以下のような事例集を公開しています。  
- https://github.com/openai/openai-cookbook
- https://platform.openai.com/docs/guides/gpt-best-practices
- https://help.openai.com/en/collections/3675942-prompt-engineering
    - https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

以下もおすすめです。  
- https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
- https://github.com/dair-ai/Prompt-Engineering-Guide
- https://github.com/f/awesome-chatgpt-prompts

##  Retrieval Augmented Generation
言語モデルへpromptingを行うことで、幅広いタスクに対応するイメージを掴みました。  
さらなる活用方法として、言語モデルの論理思考能力をもとにしたサブタスク化を活用する方法や、言語モデル自身の重みだけでなく外部のツール・モデル・情報源を活用する方法があります。  
言語モデル単体だけでタスクを行わせる場合と比べて、より難しいタスクに対応できたり、より高い精度でタスクを解かせることができると期待されます。  
このような言語モデルの活用を[Augmented Language Model](https://arxiv.org/abs/2302.07842)と呼びます。  
Augmented Language Modelの一種であるRetrieval Augmented Generationについて実装し、言語モデルの活用方法についてイメージを掴みましょう。<br>
以降は、日本語においても強いFine-TunedモデルであるLlama-3-8B-instructを使います。

In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
# https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main
# from_pretrainedの引数にモデル名を指定すると、モデルをダウンロードしてきてくれます。ダウンロードには3分ほどかかります。

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

#　メモリが足りない場合は, 量子化のために以下を有効にしてください
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct",
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

### Generation without Retrieval

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
prompt = """
以下の質問に日本語で回答してください:
質問:　東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:"""

messages = [ { "role": "user", "content": prompt} ]

input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
)
response = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

print("\n-----生成結果-----\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



-----生成結果-----
 松尾研究室の大規模言語モデル講座では、以下のような内容を扱います。

* 言語モデルの基本概念と歴史
	+ 言語モデルとは何か？その歴史と発展
	+ 主な言語モデルアルゴリズム（例：n-gram、Markov chain、recurrent neural network など）
* 言語モデルの学習と評価
	+ 言語モデルの学習方法（例：supervised learning、unsupervised learning、semi-supervised learning など）
	+ 評価指標（例：perplexity、cross-entropy、ROUGE score など）
* 大規模言語モデルの実装と評価
	+ TensorFlowやPyTorchなどの深層学習フレームワークでの実装
	+ 大規模言語モデルの評価方法（例：text classification、language translation、question answering など）
* 大規模言語モデルの応用
	+ 自然言語処理（NLP）や人工知能（AI）分野での応用
	+ 語学翻訳、文書検索、chatbot などの応用例

以上、松尾研究室の大規模言語モデル講座では、言語モデルの基本概念、学習、評価、実装、応用など、幅広い範囲の内容を扱います。


### Generation with gold passage

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
query = "東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？"
retrieved_text = """
本講座はサマースクールの一環として大規模言語モデル（Large Language Models／LLM)に関して体系的に学べる講座を目的として開催されます。
具体的には以下のスキルを身につける事を目的としています。
・LLMの原理について，事前学習からRLHFまでの一連の流れや意味合いを理解する
・LLMの学習や推論を支える技術について理解する
・すでに公開されているLLMやAPIなどの活用方法を理解する

本講座では、東京大学Deep Learning基礎講座・応用講座を公開してきた松尾・岩澤研究室が全面的に演習コンテンツを監修・作成しています。実践的な演習を通して、手を動かしながら技術を深く理解し、幅広いトピックを網羅します。
※本講座はページ下部の前提条件をご確認の上、条件が当てはまる方を対象としています。
LLMのアーキテクチャーまで取り扱う実践的な講義のため、ご自身のレベルに見合っているかご確認の上ご応募ください。
 """

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に日本語で回答してください。
質問: {query}
回答:"""

print(prompt)


本講座はサマースクールの一環として大規模言語モデル（Large Language Models／LLM)に関して体系的に学べる講座を目的として開催されます。
具体的には以下のスキルを身につける事を目的としています。
・LLMの原理について，事前学習からRLHFまでの一連の流れや意味合いを理解する
・LLMの学習や推論を支える技術について理解する
・すでに公開されているLLMやAPIなどの活用方法を理解する

本講座では、東京大学Deep Learning基礎講座・応用講座を公開してきた松尾・岩澤研究室が全面的に演習コンテンツを監修・作成しています。実践的な演習を通して、手を動かしながら技術を深く理解し、幅広いトピックを網羅します。
※本講座はページ下部の前提条件をご確認の上、条件が当てはまる方を対象としています。
LLMのアーキテクチャーまで取り扱う実践的な講義のため、ご自身のレベルに見合っているかご確認の上ご応募ください。
 
上記の文章に基づいて、質問に日本語で回答してください。
質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:


In [ ]:
messages = [ { "role": "user", "content": prompt} ]

input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
)
response = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

print("\n-----生成結果-----\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



-----生成結果-----
 東京大学の松尾・岩澤研究室が開講する大規模言語モデル講座では、LLMの原理、学習や推論を支える技術、すでに公開されているLLMやAPIなどの活用方法について、体系的に学べる講座を目的としています。具体的には、事前学習からRLHFまでの一連の流れや意味合いを理解すること、LLMの学習や推論を支える技術について理解すること、すでに公開されているLLMやAPIなどの活用方法を理解することを目的としています。


### Retrieval の実装
大規模モデルに質問をしただけでは事実に基づいた正しい回答が得られませんでしたが、回答の根拠となる文章が与えられると、根拠に基づいた回答がなされることを確認できました。

質問に応じて根拠となる文章を検索し、回答の際に参照することによって文を生成するのがRetrieval Augmented Generationです。  

どのように回答の根拠となる文章を選択するかにはさまざまな手法があります。  
今回は文章をベクトル化するモデルを使用し、ベクトルを元に質問文と記事中の文章の類似度を計算し、類似度が高い文章を選択することで回答の根拠となる文章を選択します。  <br>

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.1 MB/s eta 0:00:00


In [ ]:
# 文章や画像のembeddingを扱うためのライブラリ、HuggingFaceのモデルを使うこともできます。
from sentence_transformers import SentenceTransformer
# https://huggingface.co/Alibaba-NLP/gte-large-en-v1.5
emb_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# 文章(クエリ/質問)を1024次元のベクトルに変換する
query_embeddings = emb_model.encode("東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？", normalize_embeddings=True)
print(query_embeddings.shape)
print(query_embeddings)

(1024,)
[ 0.01382616 -0.01785062  0.00038656 ...  0.01860121 -0.01050127
 -0.01167206]


In [ ]:
# ドキュメント(文書)
input_texts = [
    "本講座はサマースクールの一環として大規模言語モデル（Large Language Models／LLM)に関して体系的に学べる講座を目的として開催されます。具体的には以下のスキルを身につける事を目的としています。・LLMの原理について，事前学習からRLHFまでの一連の流れや意味合いを理解する・LLMの学習や推論を支える技術について理解する・すでに公開されているLLMやAPIなどの活用方法を理解する。本講座では、東京大学Deep Learning基礎講座・応用講座を公開してきた松尾・岩澤研究室が全面的に演習コンテンツを監修・作成しています。実践的な演習を通して、手を動かしながら技術を深く理解し、幅広いトピックを網羅します。",
    "本講座は、学生を対象とした「深層生成モデル」に特化した全7回のセミナーです。生成モデルの基礎から始めて、最新の拡散モデルを含めたさまざまな深層生成モデルについて体系立てて学ぶことができます。今後のAI（人工知能）技術の発展において重要とされる「世界モデル」も1回分の講義として扱います。深層生成モデルは、現在注目されている「生成AI」の核心技術であり、本講座を受講することで、今後のAI技術の鍵となるトピックを詳しく学ぶことができます。",
    "深層学習を用いた強化学習に特化した全7回の講義です。東京大学でDeep Learning基礎講座を運営してきた松尾・岩澤研究室が、深層学習の基礎を習得済みの学生を対象として開講するものです。本講座では、深層学習を用いた強化学習の研究・社会実装のスタートラインに立てるレベルの知識・実装力の習得を目指します。マルコフ決定過程における価値反復法や方策勾配法、モデルベース強化学習などの強化学習の基礎的なアルゴリズムから、sim2real、模倣学習、Control as Inference、世界モデルまで、深層強化学習の種々のトピックをカバーしています。講義のみでなく、実践的な演習を通して、効率的に強化学習について手を動かしながら技術を深く理解することができます。"
    ]
# ドキュメントを1024次元のベクトルに変換する
passage_embeddings = emb_model.encode(input_texts, normalize_embeddings=True)
print(passage_embeddings.shape)

# 類似度の計算 (クエリとドキュメントの類似度)
scores = (query_embeddings @ passage_embeddings.T) * 100
print('各ドキュメントのスコア:\n', scores.tolist())
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
# 類似度の高い順のインデックスを取得
print('\n最もスコアの高いドキュメントのindex:\n', scores.argsort()[::-1][0])
# 一番高い類似度の文章を取得
print('\n最もスコアの高いドキュメント\n', input_texts[scores.argsort()[::-1][0]])

(3, 1024)
各ドキュメントのスコア:
 [73.88214874267578, 63.29391098022461, 73.61793518066406]

最もスコアの高いドキュメントのindex:
 0

最もスコアの高いドキュメント
 本講座はサマースクールの一環として大規模言語モデル（Large Language Models／LLM)に関して体系的に学べる講座を目的として開催されます。具体的には以下のスキルを身につける事を目的としています。・LLMの原理について，事前学習からRLHFまでの一連の流れや意味合いを理解する・LLMの学習や推論を支える技術について理解する・すでに公開されているLLMやAPIなどの活用方法を理解する。本講座では、東京大学Deep Learning基礎講座・応用講座を公開してきた松尾・岩澤研究室が全面的に演習コンテンツを監修・作成しています。実践的な演習を通して、手を動かしながら技術を深く理解し、幅広いトピックを網羅します。


それでは、[松尾研のニュース記事](https://weblab.t.u-tokyo.ac.jp/news/)を回答根拠として質問に答える一連の流れを実装してみましょう。  

### Document(文書)の用意 (チャンキング: Chunking) (補足)
RAGシステムを実装する上で、重要な工程の一つがドキュメントの準備です。例えば、Wikipediaの情報をドキュメントソースとして活用する場合を考えてみましょう。ウェブページ上の文章全体をそのまま使用することは、以下の理由から望ましくありません:
* APIの文字数制限:
多くの言語モデルAPIには入力文字数の上限があります。
* Retrieval（検索）時の性能:
大量の未整理テキストは、関連情報の検索精度を低下させる可能性があります。
* 意味的なノイズ:
不要な情報や文脈と無関係なデータが含まれることで、モデルの出力品質が低下する恐れがあります。

これらの課題に対処する方法として「チャンキング(Chunking)」と呼ばれるものがあります。「チャンキング」では、文章をいくつかの断片に分割し、構造化することで効率的かつ高精度にRAGを行えるようにします。「チャンキング」には主に以下の手法があります。
* 文字数や単語数による分割
* 段落や章などの文書構造に基づく分割
* 意味的なまとまりを考慮した分割

今回は二つ目の「段落や章などの文書構造に基づく分割」を行うこととします。

In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup

data_list = []

response = requests.get("https://weblab.t.u-tokyo.ac.jp/news/")
soup = BeautifulSoup(response.text, 'html.parser')
wf_cells = soup.find_all('div', class_='wp-block-column is-vertically-aligned-center is-layout-flow wp-block-column-is-layout-flow')
# print(wf_cells)

for wf_cell in wf_cells:
  data_name = wf_cell.find('p', class_='newslist-pot-title wp-block-post-title has-normal-font-size').find('a').text
  print(data_name)
  url = wf_cell.find('a')['href']
  # URLにアクセスして記事の内容を取得
  article_response = requests.get(url)
  article_html = article_response.text
  article_soup = BeautifulSoup(article_html, 'html.parser')

  entry_content = article_soup.find('div', class_='entry-content')
  if entry_content:
      paragraphs = entry_content.find_all('p')
      article_text = '\n'.join([p.get_text().strip() for p in paragraphs])
  else:
      article_text = "No content available."
      continue
  text_splits = re.split(r'\n{2,}', article_text)
  text_id = 0
  for text_split in text_splits:
      text_split = text_split.replace('\n', '')
      # if len(text_split) < 3 or len(text_split) > 300:
      #     continue
      article_info = {
          "url": url,
          "title": data_name,
          "content": text_split,
          "text_id": text_id
      }
      data_list.append(article_info)
      text_id += 1
  time.sleep(1)

東京大学松尾・岩澤研究室とGoogle、大規模言語モデルの研究開発および実装人材の育成を目的に「LLM 寄付講座」を設置
株式会社Deepcraftを松尾研発スタートアップ®︎に認定しました
当研究室の研究発表が2024年度 人工知能学会全国大会（第38回）で優秀賞を受賞しました。
政府の「新しい資本主義実現会議」（第３０回）が開催され、松尾教授が委員として参加しました。
当研究室松尾教授のコメントが、日経ビジネス電子版に掲載されました。
当研究室の論文がNeurIPS 2024に２件採録されました。
当研究室の論文がEMNLP 2024 (Main)に採録されました。
LLM 大規模言語モデル講座 2023コンテンツ 公開
2024年Aセメスター「ディープテック起業実践演習」10/2まで申込み受付中！
GCI 2024 Winter の募集開始


In [ ]:
data_list[:3]

[{'url': 'https://weblab.t.u-tokyo.ac.jp/news/2024-11-21/',
  'title': '東京大学松尾・岩澤研究室とGoogle、大規模言語モデルの研究開発および実装人材の育成を目的に「LLM 寄付講座」を設置',
  'content': '東京大学大学院工学系研究科技術経営戦略学専攻 松尾・岩澤研究室（以下「松尾研」）は、大規模言語モデル（LLM）の加速度的増加・発展を受け、2024年10月にGoogleの寄付により2027年9月までの期間で「LLM寄付講座」を設置しました。松尾研ではこれまでWeblab-10Bの開発やGENIACプロジェクトでのTanuki-8×8Bといった大規模モデルの開発、またプロンプティング（[1]）、LLMの多言語能力の理解（[2]）、Safety（[3]）、ロボティクスでの応用研究（[4]）などさまざまな研究開発を行ってきました。また、2023年度はLLMに関する全7回の講座を開講し、約2000名に講義を提供しました。2024年度は「Safety」「LLMの分析と理論「Domain Specific LLM」「LLM for Control」等のテーマを新規追加の上、全12回に講義を拡張し、受講申込み者約4000名を対象に講義を提供しています。本寄付講座では、これまでの取り組みを更に拡大し、LLMの領域特化モデルの開発やLLMのシステムへの統合、公平性を始めとした社会的影響に関する研究開発、また最先端の技術を理解し活用できる実践的な教育方法の開発と提供を加速します。本講座の設置に伴い、LLMに関する研究開発を行う研究者およびエンジニアを追加募集します。▼松尾・岩澤研究室採用HP▼各種求人ページLLM研究者：https://herp.careers/v1/weblab/j2i2oZtd_tBqLLM開発エンジニア：https://herp.careers/v1/weblab/1x8p6tXgtCyC【担当教員】特任教授\u3000松尾 豊（技術経営戦略学専攻）特任教授\u3000坂田 一郎（技術経営戦略学専攻）特任准教授\u3000岩澤 有祐（技術経営戦略学専攻）特任准教授\u3000吉田 塁（国際工学教育推進機構）【LLMに関するこれまでの取り組み】初めて開催した大規模言語モデル

### Retrieval Augmented Generation
ニュースページの更新に伴ってスクレイピングによって取得されるドキュメントの中身が変動するので, こちらが用意したqueryに回答するためのソースが含まれないことがあります。

data_listの中身を見て, 都度queryを変える等の対応をしてください。

In [ ]:
query = "東京大学松尾・岩澤研究室 GENIACプロジェクトにおいて開発したモデルは何か？"
query_embeddings = emb_model.encode([query], normalize_embeddings=True)

input_texts = [str(data['content']) for data in data_list]
passage_embeddings = emb_model.encode(input_texts, normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
# 上位3件を表示
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(input_texts[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(input_texts[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(input_texts[scores[0].argsort()[::-1][2]])

score:  83.28469
東京大学大学院工学系研究科 松尾・岩澤研究室 広報担当E-mail：pr@weblab.t.u-tokyo.ac.jp
score:  83.08865
東京大学大学院工学系研究科技術経営戦略学専攻 松尾・岩澤研究室（以下「松尾研」）は、経済産業省及び国立研究開発法人新エネルギー・産業技術総合開発機構（NEDO）が推進する、国内の生成AIの開発力を強化するためのプロジェクト「GENIAC（Generative AI Accelerator Challenge）」（注1）において、大規模言語モデル「Tanuki-8×8B」を開発し、公開いたしました。
score:  75.260475
〇関連情報：本モデルの開発過程やナレッジは、下記プロジェクトページやZennブログ記事にてオープンに公開されております。詳細は下記をご覧ください。GENIAC 松尾研LLM開発プロジェクト特設ページ：https://weblab.t.u-tokyo.ac.jp/geniac_llm/Zennブログ記事：https://zenn.dev/p/matsuolab


In [ ]:
query = "東京大学松尾・岩澤研究室 GENIACプロジェクトにおいて開発したモデルは何か？"
query_embeddings = emb_model.encode([query], normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

top_k = 2
top_k_idx = scores[0].argsort()[::-1][:top_k]


retrieved_text = f"""
* {input_texts[top_k_idx[0]]}
* {input_texts[top_k_idx[1]]}
"""

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に日本語で回答してください。
質問: {query}
回答:"""
print(prompt)


* 東京大学大学院工学系研究科 松尾・岩澤研究室 広報担当E-mail：pr@weblab.t.u-tokyo.ac.jp
* 東京大学大学院工学系研究科技術経営戦略学専攻 松尾・岩澤研究室（以下「松尾研」）は、経済産業省及び国立研究開発法人新エネルギー・産業技術総合開発機構（NEDO）が推進する、国内の生成AIの開発力を強化するためのプロジェクト「GENIAC（Generative AI Accelerator Challenge）」（注1）において、大規模言語モデル「Tanuki-8×8B」を開発し、公開いたしました。

上記の文章に基づいて、質問に日本語で回答してください。
質問: 東京大学松尾・岩澤研究室 GENIACプロジェクトにおいて開発したモデルは何か？
回答:


In [ ]:
messages = [ { "role": "user", "content": prompt} ]

input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
)
response = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

print("\n-----生成結果-----\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



-----生成結果-----
 松尾・岩澤研究室は、GENIACプロジェクトにおいて大規模言語モデル「Tanuki-8×8B」を開発しました。


どのような塊で回答根拠となるデータを蓄積するか、どのように回答根拠を選ぶか、どのように回答を生成するか、というそれぞれのステップで更なる工夫の余地があります。  
以下に参考資料を挙げておきます。
- https://github.com/openai/chatgpt-retrieval-plugin
    - https://github.com/openai/chatgpt-retrieval-plugin#limitations
    - https://github.com/openai/chatgpt-retrieval-plugin#future-directions
- https://techcommunity.microsoft.com/t5/azure-ai-services-blog/revolutionize-your-enterprise-data-with-chatgpt-next-gen-apps-w/ba-p/3762087
    - https://github.com/Azure-Samples/azure-search-openai-demo
- https://acl2023-retrieval-lm.github.io/

また、MTEB Leaderboardには性能別にEmbedding modelが順位付けされています。こちらを参考に良いRetrieval性能を持つモデルを採用することをお勧めします。
- https://huggingface.co/spaces/mteb/leaderboard

### Reranking(補足)
RAGにおいて用いられることが多いRerankingと呼ばれる手法について紹介します。
一般的に、選択された文書は質問に関係なかったり、回答に役立たないことが多くあります。したがってRerankingでは2回文書の選択を行うことで、取得する文書の精度を高めます。具体的には次のとおりです。
1. 初めに、top-kなどで関連する文書だけを多く取得する。
2. 次に、計算コストは高いが精度の良いモデルを用いて取得された文書の中から再度文書を選択する。</br>

多くの場合、選択するソースは膨大なことが多いので初めから精度の高いモデルを用いると計算コストやメモリを多く消費することになるため、1段階目で簡単に候補を絞るなどといった工夫になります。


In [ ]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn


In [ ]:
# 文章や画像のembeddingを扱うためのライブラリ、HuggingFaceのモデルを使うこともできます。
from sentence_transformers import SentenceTransformer
# https://huggingface.co/dunzhang/stella_en_1.5B_v5
rerank_model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/174k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

In [ ]:
query = "東京大学松尾・岩澤研究室 GENIACプロジェクトにおいて開発したモデルは何か？"
query_embeddings = emb_model.encode([query], normalize_embeddings=True)
passage_embeddings = emb_model.encode(input_texts, normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
top_k = 5
top_k_idx = scores[0].argsort()[::-1][:top_k]
retrieve_item = [input_texts[idx] for idx in top_k_idx]
print("\n\n-------文書の取得(1回目)-------")
for item in retrieve_item:
    print("・", item)

#Reranking
passage_embeddings = rerank_model.encode(retrieve_item, normalize_embeddings=True)
query_embeddings = rerank_model.encode([query], normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
top_k = 2
top_k_idx = scores[0].argsort()[::-1][:top_k]
retrieved_text = f"""
{retrieve_item[top_k_idx[0]]}

{retrieve_item[top_k_idx[1]]}
"""

print("\n\n-------文書の取得(2回目)-------")
for item in [retrieve_item[top_k_idx[0]], retrieve_item[top_k_idx[1]]]:
    print("・", item)



-------文書の取得(1回目)-------
・ 東京大学大学院工学系研究科 松尾・岩澤研究室 広報担当E-mail：pr@weblab.t.u-tokyo.ac.jp
・ 東京大学大学院工学系研究科技術経営戦略学専攻 松尾・岩澤研究室（以下「松尾研」）は、経済産業省及び国立研究開発法人新エネルギー・産業技術総合開発機構（NEDO）が推進する、国内の生成AIの開発力を強化するためのプロジェクト「GENIAC（Generative AI Accelerator Challenge）」（注1）において、大規模言語モデル「Tanuki-8×8B」を開発し、公開いたしました。
・ 〇関連情報：本モデルの開発過程やナレッジは、下記プロジェクトページやZennブログ記事にてオープンに公開されております。詳細は下記をご覧ください。GENIAC 松尾研LLM開発プロジェクト特設ページ：https://weblab.t.u-tokyo.ac.jp/geniac_llm/Zennブログ記事：https://zenn.dev/p/matsuolab
・ 松尾・岩澤研では，これまで深層生成モデルや深層強化学習といった基礎アルゴリズムの開発，画像認識，自然言語処理，ロボティクスに関する研究，それらの技術的コアになる世界モデルに関する研究などの研究を行ってきました．現在はフルタイムの研究員の数も10名を超え，2023年度にはICML，ICLR，EMNLP，NAACL，ICRAなどのトップ会議，トップジャーナルに合わせて14本の論文が採択されています．（最近の採択論文一覧はこちら）．最近ではBrain-Inspired Intelligenceチームも発足し，深層学習の限界を根本的に解消することを目指した研究開発も行っています．
・ 株式会社アマゾンで働かれているDanushka先生が7/31（水）に松尾研を訪問されました。松尾教授が所属していた石塚研究室にDanushka先生も所属されており、松尾教授とは古くから関わりがありました。そうしたご縁から、例年ご講演をいただいています。当日は松尾研の研究員や配属学生以外にも、講義受講生などオンラインの参加者含め100名近くの方が参加し、「空気を読むLLM」というテーマで講演いただきました。


-------文書の取得(2回目)-------

### Tanuki 8B(補足)
経産省及びNEDOが進める日本国内の生成AI基盤モデル開発を推進する「GENIAC」プロジェクトにおいて、松尾・岩澤研究室が開発・公開した「Tanuki-8×8B」の軽量版である、「Tanuki-8B」を動かしてみたい方に向けた補足となります。<br>
ここではJapanese MT-benchの質問を例に試してみます。
https://huggingface.co/weblab-GENIAC/Tanuki-8B-dpo-v1.0

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model = AutoModelForCausalLM.from_pretrained("weblab-GENIAC/Tanuki-8B-dpo-v1.0", device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("weblab-GENIAC/Tanuki-8B-dpo-v1.0")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

messages = [
    {"role": "system", "content": "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。"},
    {"role": "user", "content": "経済成長率、消費者物価指数、失業率などの経済指標と日本銀行の金融政策との関係性を説明してください。その経済指標に影響を与える主な政策手段を示し、それぞれの効果について考察してください。"}
]

input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
output_ids = model.generate(input_ids,
                            max_new_tokens=128,
                            temperature=0.5,
                            streamer=streamer)
result = output_ids[0][input_ids.shape[-1]:]
output = tokenizer.decode(result, skip_special_tokens=True)
print("\n-----生成結果-----\n", output.strip())

del input_ids

model-00001-of-00004.safetensors:  10%|9         | 493M/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/533M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


日本銀行（中央銀行である日銀）の金融政策は、経済成長率、消費者物価指数（CPI）、失業率といった主要な経済指標に密接に関連しています。これらの指標は相互に影響し合い、経済全体の健康状態やインフレーション、雇用状況を反映します。以下にそれぞれの指標と日銀の政策手段、そしてその効果について詳しく説明します。

まず、経済成長率はGDPの成長率として表され、企業の生産活動、消費、投資など多く

-----生成結果-----
 日本銀行（中央銀行である日銀）の金融政策は、経済成長率、消費者物価指数（CPI）、失業率といった主要な経済指標に密接に関連しています。これらの指標は相互に影響し合い、経済全体の健康状態やインフレーション、雇用状況を反映します。以下にそれぞれの指標と日銀の政策手段、そしてその効果について詳しく説明します。

まず、経済成長率はGDPの成長率として表され、企業の生産活動、消費、投資など多く


### 参考文献


*   Grégoire Mialon, Roberto Dessì, Maria Lomeli, Christoforos Nalmpantis, Ram Pasunuru, Roberta Raileanu, Baptiste Rozière, Timo Schick, Jane Dwivedi-Yu, Asli Celikyilmaz, Edouard Grave, Yann LeCun, Thomas Scialom.      Augmented Language Models: a Survey, 2023, https://arxiv.org/abs/2302.07842
*   Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Brian Ichter, Fei Xia, Ed Chi, Quoc Le, Denny Zhou. Chain-of-Thought Prompting Elicits Reasoning in Large Language Models, 2022, https://arxiv.org/abs/2201.11903
* yahoojapan JGLUE: Japanese General Language Understanding Evaluation, https://github.com/yahoojapan/JGLUE, アクセス日: 2024/08/13

